---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def blight_model():
    '''
    Builds a machine learning model that predicts if a
    blight ticket will be paid. The dataset contains
    27 features and its target is compliance. Compliance
    refers to whether a blight ticket was paid.
    '''
    def x_y_split(df):
        ''' takes DataFrame as input and 
            splits it into the X and y data
        
        df = the input pd.DataFrame
        
        not_features = list of column names that will not 
        be used as features because they are not included
        in the test data and/or because they are leaky features
        '''
        
        not_features =['payment_amount', 'payment_date', 'payment_status',
               'balance_due', 'collection_status', 'compliance',
               'compliance_detail']
        
        return df.loc[:, ~df.columns.isin(not_features)], df['compliance']
    
    def data_cleaning(df, categories):
        '''
        remove 'not responsible' entries from data
        clean up violator names
        clean up city
        '''
        df = df.loc[df.loc[:,'compliance'].notna()] # May need to leave these here?
        df['city'] = df.loc[:,'city'].str.upper().str.rstrip()

        df['city'].replace('\w([\s]{2,})\w|,$', '', regex = True, inplace = True)
        df['city'].replace('^DET.*|DEETROIT', 'DETROIT', regex = True, inplace = True)
        df['city'].replace('\s(HIGHT|HGHTS?|HGTS|HGT|HTS|HEIGHTS|HEIGHTS\'S|HGT\'S|HEGHT|HT|HEIGTS|HEIGHT|HEIGTHS|HIEGHTS|HYS|HEIGHS|HEIGHTS ST|HEIGHTSDET|HEIHTS|GHHTS).?$',
                                   ' HEIGHTS', regex = True, inplace = True)
        df['city'].replace('^(SERLING|STELING|STERING|SWTERLING|STERLINGHT|STERLINGT|SSTERLING|STERLINGS|STEERLING|STERLNG|STERLIN|STERLINGHT|STEERLING|STERLINGG|STERLINGT)\s',
                                   'STERLING ', regex = True, inplace = True)
        df['city'].replace('^(STERLINGHEIGHTS|STERLING DR|STERLINGEIGHTS|STERLINGEIGHTS|STERLINGT?|STERLINGHGHTS|STERLING HIGH|STERLING HILLS)$',
                                   'STERLING HEIGHTS', regex = True, inplace = True)
        df['city'].replace('(TWP|TWNSHIP).?', 'TOWNSHIP', regex = True, inplace = True)
        df['city'].replace('W.?\s?BLOOMFIELD', 'WEST BLOOMFIELD', regex=True, inplace = True)
        df['city'].replace('W.?\s?PALM BEACH', 'WEST PALM BEACH', regex=True, inplace = True)

        df['city'].replace('SFLD.?', 'SOUTHFIELD', regex=True, inplace = True)
        df['city'].replace('(HIILS|HILLS|HLLS).?$', 'HILLS', regex=True, inplace = True)
        df['city'].replace('^GROSSE? ?(PI?O?I?I?N?TE?E?S?D?.?|PT|IS?L?LE?|PA?R?K|OTE|POI9NTE|POI?NITE)? ?(FARMS?|SHORE|W?O?O?DS|PA?R?K|OINTAR)?',
                                   'GROSSE POINTE', regex=True, inplace = True)
        df['city'].replace('RALIEGH', 'RALEIGH', regex=True, inplace = True)
        df['city'].replace('SOUHFIELD|STHFLD.|SOUTHFIELD #460', 'SOUTHFIELD', regex=True, inplace = True)
        df['city'].replace('PHILADEPHIA', 'PHILADEPHIA', inplace = True)
        df['city'].replace('FARMINGTONHILLS', 'FARMINGTON HILLS', inplace = True)
        
        df.loc[(~df['violator_name'].isnull() &
        df['violator_name'].str.contains('ACORN', case = False)), 'violator_name'] = 'ACORN INVESTMENT CO.'
        df.loc[(~df['violator_name'].isnull() &
        df['violator_name'].str.contains('DEUTSCHE',case = False)), 'violator_name'] = 'DEUTSCHE BANK'
        df.loc[(~df['violator_name'].isnull() &
        df['violator_name'].str.contains('WELLS?',case = False) &
        df['violator_name'].str.contains('FARGO',case = False)), 'violator_name'] = 'WELLS FARGO BANK'
        df.loc[(~df['violator_name'].isnull() &
        df['violator_name'].str.contains('MORGAN',case = False) &
        df['violator_name'].str.contains('CHASE',case = False) &
        df['violator_name'].str.contains('J.?P.?',case = False)), 'violator_name'] = 'JP MORGAN CHASE BANK'
        
        df['ticket_issued_date'] = pd.to_datetime(df['ticket_issued_date'])
        df['ticket_issued_day_of_week'] = df['ticket_issued_date'].apply(lambda x: x.weekday())
        df['ticket_issued_month'] = df['ticket_issued_date'].apply(lambda x: x.month)
        
        df['hearing_date'] = pd.to_datetime(df['hearing_date'].fillna('1980-01-01 00:00:00'))
        df['hearing_year'] = df['hearing_date'].apply(lambda x: x.year)
        df['hearing_month'] = df['hearing_date'].apply(lambda x: x.month)
        df['hearing_day_of_week'] = df['hearing_date'].apply(lambda x: x.weekday())
        
        for cat in categories:
            df['_'.join([cat, 'category'])] = df[cat].fillna('<unknown>').astype('category').cat.codes
        
        return df

    ############
    ############
    
    wanted_features = [#'violator_name',
#                        'city',
#                        'inspector_name',
#                        'agency_name',
#                        'violation_code',
#                        'fine_amount',
#                        'discount_amount',
                       'admin_fee',
                       'state_fee',
                       'late_fee',
#                        'clean_up_cost',
#                        'ticket_issued_day_of_week',
#                        'ticket_issued_month',
                       'hearing_year',
                       'hearing_month', 
                       'hearing_day_of_week',
                     'compliance'] # Always include compliance
    category_columns = ['violator_name', 'city', 'inspector_name', 'agency_name', 'violation_code']
    wanted_columns = ['_'.join([feat, 'category']) if feat in category_columns else feat for feat in wanted_features]
    
    # Load, clean & split training data
    df = pd.read_csv('train.csv', encoding = 'ISO-8859-1', 
                 dtype = {'zip_code': str,       # 11
                          'non_us_str_code':str, # 12
                          'grafitti_status':str  # 31
                         }
                )
    
    df = data_cleaning(df, category_columns).loc[:,wanted_columns]
    
    X, y = x_y_split(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    
    # free memory by removing df
    del df
    
#     # Train the model
    model = LogisticRegression(penalty='l2', C=0.45,
                               solver='lbfgs',
                              max_iter = 1000).fit(X_train, y_train)
    
#     # Load, clean & split testing data
#     X_test = pd.read_csv('test.csv', encoding = 'ISO-8859-1',
#                          dtype = {'zip_code': str,  # 11
#                                   'non_us_str_code':str, # 12
#                                   'grafitti_status':str  # 31
#                                  }
#                         )
    # Predict y values using testing data
    y_pred = model.predict_proba(X_test)# model.predict(X_test)
    
#     fpr, tpr, thresh = roc_curve(y_test, y_pred.T[1])
#     auc = roc_auc_score(y_test, y_pred.T[1])
    
#     %matplotlib notebook
#     plt.figure()
#     plt.plot(fpr, tpr, label=f'area={auc:0.3f}')
#     plt.plot([0,1],[0,1],'r--')
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.legend(loc='lower right')
#     plt.show
    
    return pd.Series(y_pred.T[1])#auc


# blight_model()

In [ ]:
# blight_model()

# Exploratory Data Analysis

In [9]:
# Comment this cell out---or convert to markdown cell---
# before submitting for evaluation.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# The data need to be loaded with ISO-8859-1 encoding
# or an error is thrown.
# The dtype for columns 11, 12 & 31 need to be specified
# I have chosen str as a dtype for all three after a
# cursory glance at these columns.
df = pd.read_csv('train.csv', encoding = 'ISO-8859-1', 
                 dtype = {'zip_code': str,       # 11
                          'non_us_str_code':str, # 12
                          'grafitti_status':str  # 31
                         }
                )

Explore feature names & feature names that need to change for clarify.

In [10]:

# These are potentially leaky features and will be ignored
not_features =['payment_amount', 'payment_date', 'payment_status',
               'balance_due', 'collection_status', 'compliance',
               'compliance_detail']

# Print shape of training data (features only)
print(f'The shape of the feature training data: {df.loc[:,~df.columns.isin(not_features)].shape}.\n')


#     Split X_train from y_train
X_train = df.loc[:, ~df.columns.isin(not_features)]
y_train = df['compliance']

# Count the number of NaN in each column
# From this we can glean that:
# violator_zip_code is empty -> drop this column
# mailing non_us_str_code only has a few values -> drop this column
# grafitti_status only has one entry -> drop this column
X_nan = X_train.isna().sum()
print('The number of NaN rows for each column:')
print(X_nan)
print('\n')

# Assuming 0 is noncompliant and 1 is compliant
print('A summary of the y_train vector:')
print(y_train.value_counts())
print(f'number of None or NaN in y_train: {y_train.isna().sum()}.')
print('\n')

# These three columns gave a dtype warning when using pandas.read_csv
# The dtype will be specified for these (I have chosen str for all
# three columns)
print('specify a dtype for the following columns:')
for col in df.columns[[11,12,31]]:
    print(f'\t{col}')
print('\n')

# Rename some of the features for clarity
# feature_name_change = {'city':'mailing_city',
#                        'state': 'mailing_state',
#                        'zip_code': 'mailing_zip_code',
#                        'non_us_str_code': 'mailing_non_us_str_code',
#                        'country': 'mailing_country'}
# df.rename(columns = feature_name_change, inplace = True)

# List all potential features:
print(f'The potential features are:')
for feat_name in df.loc[:,~df.columns.isin(not_features)].columns:
    print(f'\t{feat_name}')
print('\n')

print(df['disposition'].value_counts())
print('\n')

print(df['hearing_date'].value_counts(dropna=False))

# with pd.option_context('display.max_rows', None):
#     print('non NaN values in grafitti_status')
#     display(X_train['grafitti_status'].loc[X_train['grafitti_status'].notna()])

The shape of the feature training data: (250306, 27).

The number of NaN rows for each column:
ticket_id                          0
agency_name                        0
inspector_name                     0
violator_name                     34
violation_street_number            0
violation_street_name              0
violation_zip_code            250306
mailing_address_str_number      3602
mailing_address_str_name           4
city                               0
state                             93
zip_code                           1
non_us_str_code               250303
country                            0
ticket_issued_date                 0
hearing_date                   12491
violation_code                     0
violation_description              0
disposition                        0
fine_amount                        1
admin_fee                          0
state_fee                          0
late_fee                           0
discount_amount                    0
clean_up_cost    

In [ ]:
# Look at violators that are not in the USA
# Some of the Canadian zip codes are not
# valid zip codes in Canada.
# print(df[['mailing_zip_code','mailing_country']].loc[df['mailing_country'] != 'USA'])
# print('\n\n')
# print('######')
feature_names = df.loc[:, ~df.columns.isin(not_features)].columns

# A quick look at city names reveals some entries need to be cleaned up.
# There are several cities showing up in different entries
# because the cases are different (e.g., DETROIT, Detroit, detroit).
# Detroit is also entered as DET and DET.
# W. BLOOMFIELD & WEST BLOOMFIELD

df['city'] = df['city'].str.upper().str.rstrip()
df['city'].replace('\w([\s]{2,})\w|,$', '', regex = True, inplace = True)
df['city'].replace('^DET.*|DEETROIT', 'DETROIT', regex = True, inplace = True)
df['city'].replace('\s(HIGHT|HGHTS?|HGTS|HGT|HTS|HEIGHTS|HEIGHTS\'S|HGT\'S|HEGHT|HT|HEIGTS|HEIGHT|HEIGTHS|HIEGHTS|HYS|HEIGHS|HEIGHTS ST|HEIGHTSDET|HEIHTS|GHHTS).?$',
                           ' HEIGHTS', regex = True, inplace = True)
df['city'].replace('^(SERLING|STELING|STERING|SWTERLING|STERLINGHT|STERLINGT|SSTERLING|STERLINGS|STEERLING|STERLNG|STERLIN|STERLINGHT|STEERLING|STERLINGG|STERLINGT)\s',
                           'STERLING ', regex = True, inplace = True)
df['city'].replace('^(STERLINGHEIGHTS|STERLING DR|STERLINGEIGHTS|STERLINGEIGHTS|STERLINGT?|STERLINGHGHTS|STERLING HIGH|STERLING HILLS)$',
                           'STERLING HEIGHTS', regex = True, inplace = True)
df['city'].replace('(TWP|TWNSHIP).?', 'TOWNSHIP', regex = True, inplace = True)
df['city'].replace('W.?\s?BLOOMFIELD', 'WEST BLOOMFIELD', regex=True, inplace = True)
df['city'].replace('W.?\s?PALM BEACH', 'WEST PALM BEACH', regex=True, inplace = True)

df['city'].replace('SFLD.?', 'SOUTHFIELD', regex=True, inplace = True)
df['city'].replace('(HIILS|HILLS|HLLS).?$', 'HILLS', regex=True, inplace = True)
df['city'].replace('^GROSSE? ?(PI?O?I?I?N?TE?E?S?D?.?|PT|IS?L?LE?|PA?R?K|OTE|POI9NTE|POI?NITE)? ?(FARMS?|SHORE|W?O?O?DS|PA?R?K|OINTAR)?',
                           'GROSSE POINTE', regex=True, inplace = True)
df['city'].replace('RALIEGH', 'RALEIGH', regex=True, inplace = True)
df['city'].replace('SOUHFIELD|STHFLD.|SOUTHFIELD #460', 'SOUTHFIELD', regex=True, inplace = True)
df['city'].replace('PHILADEPHIA', 'PHILADEPHIA', inplace = True)
df['city'].replace('FARMINGTONHILLS', 'FARMINGTON HILLS', inplace = True)


# print('\n###')
# print(df['city'].loc[df['city'].str.contains('MACOMB')].value_counts())
# print('\n###')
# print(df['city'].loc[df['city'].str.contains('GROSSE')].value_counts())

#MACOMB  TOWNSHIP
# for feature in feature_names:
#     print(f'\nList & Frequency of {feature}')
#     print(df[feature].value_counts())
# with pd.option_context('display.max_rows', None):
#     display(df['city'].value_counts())

In [ ]:
df.loc[(~df['violator_name'].isnull()&
        df['violator_name'].str.contains('ACORN', case = False)), 'violator_name'] = 'ACORN INVESTMENT CO.'
df.loc[(~df['violator_name'].isnull()&
        df['violator_name'].str.contains('DEUTSCHE',case = False)), 'violator_name'] = 'DEUTSCHE BANK'
df.loc[(~df['violator_name'].isnull()&
        df['violator_name'].str.contains('WELLS?',case = False)&
        df['violator_name'].str.contains('FARGO',case = False)), 'violator_name'] = 'WELLS FARGO BANK'
df.loc[(~df['violator_name'].isnull()&
        df['violator_name'].str.contains('MORGAN',case = False)&
        df['violator_name'].str.contains('CHASE',case = False) &
        df['violator_name'].str.contains('J.?P.?',case = False)), 'violator_name'] = 'JP MORGAN CHASE BANK'


# with pd.option_context('display.max_rows', None):
#     display(df['violator_name'].dropna().loc[df['violator_name'].dropna().str.contains('BANK', case=False)].value_counts())
#     display(df['violator_name'].describe())
#     display(df['violator_name'].isnull().sum())
#     display(df['violator_name'].value_counts())

In [ ]:
with pd.option_context('display.max_rows',None):
    display(df['agency_name'].value_counts())

feature--pair plot

In [ ]:
from matplotlib import cm
from pandas.plotting import scatter_matrix
%matplotlib notebook

# cmap = cm.get_cmap('gnuplot')
# scatter = scatter_matrix(X_train, c = y_train,
#                          marker = 'o', s = 40,
#                          hist_kwds = {'bins':15},
#                          figsize = (12, 12),
#                          cmap = cmap)

plt.figure()
plt.plot(X_train['violator_name'].dropna(), y_train.loc[~X_train['violator_name'].isnull()], '.')
plt.xticks(rotation=90)
plt.show()

# Output Check
Provided by course assistant.

In [ ]:
bm = blight_model()
res = 'Data type Test: '
res += ['Failed: type(bm) should Series\n','Passed\n'][type(bm)==pd.Series]
res += 'Data shape Test: '
res += ['Failed: len(bm) should be 61001\n','Passed\n'][len(bm)==61001]
res += 'Data Values Test: '
res += ['Failed: all values should be in [0.,1.]\n','Passed\n'][all((bm<=1.) & (bm>=0.))]
res += 'Data Values type Test: '
res += ['Failed: bm.dtype should be float\n','Passed\n'][str(bm.dtype).count('float')>0]
res += 'Index type Test: '
res += ['Failed: type(bm.index) should be Int64Index\n','Passed\n'][type(bm.index)==pd.Int64Index]
res += 'Index values type Test: '
res += ['Failed: type(bm.index[0]) should be int64\n','Passed\n'][str(type(bm.index[0])).count("int64")>0]

res += 'Output index shape test:'
res += ['Failed, bm.index.shape should be (61001,)\n','Passed\n'][bm.index.shape==(61001,)]

res += 'Output index test: '
if bm.index.shape==(61001,):
    res +=['Failed\n','Passed\n'][all(pd.read_csv('test.csv',usecols=[0],index_col=0).sort_index().index.values==bm.sort_index().index.values)]
else:
    res+='Failed'
print(res)